In [1]:
!pip install feedparser

     |████████████████████████████████| 81kB 3.7MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-cp37-none-any.whl size=6067 sha256=0e21b4a56b1d66a5c3e34b3e545b6a978ef9d36e72ba2916619ffb4e40769073
  Stored in directory: /root/.cache/pip/wheels/f1/80/5a/444ba08a550cdd241bd9baf8bae44be750efe370adb944506a
Successfully built sgmllib3k


In [6]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import feedparser
import os
import numpy as np
from requests import Session
from multiprocessing.dummy import Pool as ThreadPool

In [7]:
s = Session()
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                         'AppleWebKit/537.36 (KHTML, like Gecko) '\
                         'Chrome/75.0.3770.80 Safari/537.36'}
# Add headers
s.headers.update(headers)

In [8]:
def get_article_items(category,pages):
  pagesToGet = pages
  data = pd.DataFrame([])
  for page in range(1,pagesToGet+1):
      url = category + '/page/' + str(page)
      #an exception might be thrown, so the code should be in a try-except block
      try:
          #use the browser to get the url. This is suspicious command that might blow up.
          page=s.get(url)                             # this might throw an exception if something goes wrong.
      except Exception as e:                                   # this describes what to do if an exception is thrown
          error_type, error_obj, error_info = sys.exc_info()      # get the exception information
          print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
          print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
          continue                                              #ignore this page. Abandon this and go back. 
      soup=BeautifulSoup(page.text,'lxml')
      links = soup.find_all('li',attrs={"class":"lx-stream__post-container"})
      for i in links:
          try: 
              title = i.find("span",attrs={'class':'lx-stream-post__header-text gs-u-align-middle'}).text.strip()
              link = "https://www.bbc.com"
              link += i.find('a',attrs={"class":"qa-heading-link lx-stream-post__header-link"})["href"]
              date = i.find('div',attrs={"class":"gs-u-display-flex"}).find("span",attrs={"class":"qa-post-auto-meta"}).text
              data = data.append(pd.DataFrame({'title': title, 'url': link,'date' : date}, index=[0]), ignore_index=True)
          except:
              continue
  return data
    

    

In [9]:
data = pd.DataFrame([])
list1 = []

In [10]:
%%time
Health = get_article_items("https://www.bbc.com/turkce/topics/cnq68n6wgzdt",100)
Health["category"] = "Health"
list1.append(Health)

CPU times: user 9.57 s, sys: 253 ms, total: 9.83 s
Wall time: 1min 32s


In [11]:
Health.head()

,title,url,date,category
0,İngiltere-Türkiye arasında seyahat için mevcut...,https://www.bbc.com/turkce/haberler-dunya-5632...,18:12,Health
1,Türkiye'de son 24 saatte Covid vaka sayısı 61 ...,https://www.bbc.com/turkce/haberler-turkiye-51...,22:10 20 Nisan 2021,Health
2,"Sağlık Bakanlığı: 61 bini aşkın yeni vaka, 346...",https://www.bbc.com/turkce/live/haberler-turki...,11:05 20 Nisan 2021,Health
3,Covid-19 hastalığını atlatanlar ne kadar süred...,https://www.bbc.com/turkce/haberler-dunya-5234...,9:55 20 Nisan 2021,Health
4,"İsrail'de yapılan araştırmalar, Pfizer aşısını...",https://www.bbc.com/turkce/haberler-dunya-5600...,7:49 20 Nisan 2021,Health


In [12]:
%%time
technology = get_article_items("https://www.bbc.com/turkce/topics/c2dwqnwkvnqt",100)
technology["category"] = "technology"
list1.append(technology)

CPU times: user 7.33 s, sys: 187 ms, total: 7.51 s
Wall time: 2min 3s


In [15]:
technology.head()

,title,url,date,category
0,"Apple son model iPad, iMac ve AirTag’leri tanı...",https://www.bbc.com/turkce/haberler-dunya-5683...,9:29,technology
1,"Facebook, Clubhouse benzeri Live Audio Rooms v...",https://www.bbc.com/turkce/haberler-dunya-5681...,9:25 20 Nisan 2021,technology
2,Elon Musk: 2 kişinin öldüğü Tesla kazasında 'o...,https://www.bbc.com/turkce/haberler-dunya-5681...,2:01 20 Nisan 2021,technology
3,Dogecoin: Şaka amaçlı geliştirilen ve piyasa d...,https://www.bbc.com/turkce/haberler-dunya-5593...,12:38 19 Nisan 2021,technology
4,Elektrikli tırlar ‘dizellerle rekabet edebilir’,https://www.bbc.com/turkce/haberler-dunya-5671...,13:31 11 Nisan 2021,technology


In [16]:
%%time
science = get_article_items("https://www.bbc.com/turkce/topics/c404v74nk56t",32)
science["category"] = "science"
list1.append(science)

CPU times: user 3.25 s, sys: 62.8 ms, total: 3.31 s
Wall time: 31.9 s


In [17]:
science.head()

,title,url,date,category
0,Türkiye'de kullanılan Çin aşısı CoronaVac hakk...,https://www.bbc.com/turkce/haberler-dunya-5516...,10:26 15 Nisan 2021,science
1,Neden eğlenirken çığlık atarız?,https://www.bbc.com/turkce/haberler-dunya-5675...,4:33 15 Nisan 2021,science
2,"Yuri Gagarin, 60 yıl önce ilk insanlı uzay uçu...",https://www.bbc.com/turkce/haberler-dunya-5671...,12:10 12 Nisan 2021,science
3,"Fizikçilere göre, doğadaki 5. temel kuvvete il...",https://www.bbc.com/turkce/haberler-dunya-5667...,17:22 8 Nisan 2021,science
4,Birinin yalan söylediğini anlamak mümkün mü?,https://www.bbc.com/turkce/haberler-dunya-5666...,2:00 8 Nisan 2021,science


In [18]:
%%time
Economy = get_article_items("https://www.bbc.com/turkce/topics/cg726y2k82dt",85)
Economy["category"] = "economy"
list1.append(Economy)

CPU times: user 9.52 s, sys: 176 ms, total: 9.7 s
Wall time: 1min 25s


In [19]:
Economy.head()

,title,url,date,category
0,Kısa çalışma ödeneği Haziran sonuna kadar uzat...,https://www.bbc.com/turkce/haberler-turkiye-53...,14:53,economy
1,"Netflix büyüme beklentilerini karşılayamadı, p...",https://www.bbc.com/turkce/haberler-dunya-5681...,10:10,economy
2,Canikli: 128 milyar doların 75 milyarı bankala...,https://www.bbc.com/turkce/haberler-turkiye-54...,14:35 20 Nisan 2021,economy
3,Kripto para yönetmeliği nedir ve kimleri etkil...,https://www.bbc.com/turkce/haberler-turkiye-56...,17:23 16 Nisan 2021,economy
4,"'128 milyar dolar' tartışması nasıl başladı, T...",https://www.bbc.com/turkce/haberler-turkiye-54...,12:47 16 Nisan 2021,economy


In [20]:
%%time
sports = get_article_items("https://www.bbc.com/turkce/topics/c340qx04vwwt",27)
sports["category"] = "sports"
list1.append(sports)

CPU times: user 2.8 s, sys: 47.7 ms, total: 2.85 s
Wall time: 27 s


In [21]:
df = Health.append(list1,ignore_index=True)

In [22]:
df = df.drop_duplicates(subset='url')

In [23]:
df.shape

(1644, 4)

In [24]:
s = Session()
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                         'AppleWebKit/537.36 (KHTML, like Gecko) '\
                         'Chrome/75.0.3770.80 Safari/537.36'}
# Add headers
s.headers.update(headers)

    
def get_user_need(page):
    try:
        article = s.get(page)
        soup = BeautifulSoup(article.content, 'lxml')
        x = [i for i in soup.find("script",text = re.compile(r'(?<="campaignName".").+?(?=\")'))][0]
        return re.search(r'(?<="campaignName".").+?(?=\")',x)[0]
    except:
        return np.nan
    
def get_text(url):
    try:
        article = s.get(url)
        soup = BeautifulSoup(article.content, 'lxml')
        body = soup.find(role='main')
        text = [p.text for p in body.find_all('p',dir="ltr")]
        return ''.join(text)
    except:
        return np.nan


In [25]:
%%time
pool = ThreadPool(5)
user_need = pool.map(get_user_need, df["url"])
df["User Need"] = user_need

CPU times: user 1min 3s, sys: 3.24 s, total: 1min 6s
Wall time: 3min 43s


In [26]:
df["User Need"] = df["User Need"].apply(lambda s: re.sub(r"(WS - )", "", s) if pd.notnull(s) else s)


In [27]:
%%time
page_text = pool.map(get_text, df["url"])
df["text"] = page_text

CPU times: user 1min 19s, sys: 19 s, total: 1min 38s
Wall time: 1min 16s


In [28]:
df["User Need"].value_counts()

Update me              781
Give me perspective    435
Educate me              85
Keep me on trend        81
Divert me               77
CS                      56
Inspire me              40
SEO Pilot                3
Name: User Need, dtype: int64

In [29]:
df.to_csv("bbc_turkish.csv",index=False)